### SSMI Decoder 

NRT

Bootstrap

In [2]:
import numpy as np
import pandas as pd

In [3]:
def decode_datafile(filename,datasource='NRT'):
    icefile = open(filename, 'rb')

    if datasource is 'NRT':
       #remove the header
       icefile.seek(300)
       ice = np.fromfile(icefile,dtype=np.uint8)
       ice[ice >= 253] = 0
       ice = ice/2.5
    elif datasource is 'Bootstrap':
       #no header
       ice = np.fromfile(icefile,dtype=np.uint16)
       ice = ice/10.
       ice[ice == 110] = 0 #110 is land
       ice[ice == 120] = 100 #120 is polar hole
    else:
       ice=np.nan

    return ice;


def get_date(filename):
    icefile = open(filename, 'rb')
    header = icefile.read(300)
    #the date is located btw the following bytes
    date = date=header[219:229].decode() #gets date from header
    date = dt.datetime.strptime(date,"%m/%d/%Y")
    return date;

def decode_latlon(filename):
    latlon_file = open(filename, 'rb')
    output = np.fromfile(latlon_file,dtype='<i4')
    output = output/100000.0
    return output;

In [4]:
fp = '/Users/bell/in_and_outbox/data_sets/nrt_nasatime_seaice/nt_20190303_f18_nrt_n.bin'
ice = decode_datafile(fp,datasource='NRT')

In [5]:
lats_fp='data/psn25lats_v3.dat'
lats = decode_latlon(lats_fp)
lons_fp='data/psn25lons_v3.dat'
lons = decode_latlon(lons_fp)

In [6]:
np.shape(ice)

(136192,)

In [7]:
np.shape(lats)

(136192,)

In [10]:
data={'latitude':decode_latlon(lats_fp), 'longitude':decode_latlon(lons_fp),
      'ice_conc':decode_datafile(fp,'NRT')}
df=pd.DataFrame(data)

In [11]:
df.describe()

,latitude,longitude,ice_conc
count,136192.000000,136192.000000,136192.000000
mean,57.624520,17.489821,15.403627
std,12.239096,104.975745,34.140085
min,31.102670,-180.000000,0.000000
25%,47.900800,-70.346180,0.000000
50%,56.940085,6.666425,0.000000
75%,66.304733,115.033140,0.000000
max,89.836820,179.813980,100.400000
